In [16]:
filepath_common = 'proturn_output_'
strains = ['aj', 'balbc', 'c57', 'cej', 'dba', 'fvb']
exp = ['ctrl', 'iso'] 
hl = 'hl.out'
dat = 'hl-data_masked'
mask = [1,2,3,4]

In [ ]:
for s in strains: 
    for e in exp: 
        for i in mask: 
            print(f'Rscript mice.R {i} {filepath_common}{s}_{e}/{hl} {filepath_common}{s}_{e}/{dat}_{i}.csv')

In [21]:
import pandas as pd

# Example DataFrames
data_true = {
    'ID': ['id1', 'id2', 'id3'],
    'value': [10, 20, 30],
    'value2': [10, 20, 30]
}
data_pred = {
    'ID': ['id1', 'id2', 'id3'],
    'value': [10, 20, 30],
    'value2': [10, 21, 30]
}

df_true = pd.DataFrame(data_true)
df_pred = pd.DataFrame(data_pred)

In [22]:
import numpy as np 
true = np.array(df_true.drop('ID', axis=1))
pred = np.array(df_pred.drop('ID', axis=1))
true, pred


(array([[10, 10],
        [20, 20],
        [30, 30]]),
 array([[10, 10],
        [20, 21],
        [30, 30]]))

In [23]:
(true - pred)**2

array([[0, 0],
       [0, 1],
       [0, 0]])

In [24]:
#sum of squared differences
np.sum((true - pred)**2)

1

In [27]:
for i in range(10):
    print(i+1)

1
2
3
4
5
6
7
8
9
10


In [12]:
import numpy as np 
import pandas as pd
data_masked = {
    'ID': ['id1', 'id2', 'id3'],
    'value': [np.nan, np.nan, 30],
    'value2': [10, 40, np.nan]
}

data_true = {
    'ID': ['id1', 'id2', 'id3'],
    'value': [10, 3, 30],
    'value2': [10, 20, 30]
}

a = pd.DataFrame(data_masked)
b= pd.DataFrame(data_true)
print(a)
#get all indices where value is null
indices = a.loc[a['value'].isnull(), 'value'].index
print(indices)

print(b.loc[indices])
np.sum(b.loc[indices, 'value'] ** 2)

    ID  value  value2
0  id1    NaN    10.0
1  id2    NaN    40.0
2  id3   30.0     NaN
Index([0, 1], dtype='int64')
    ID  value  value2
0  id1     10      10
1  id2      3      20


109

In [13]:
#keep just 2nd two columns of a
a.iloc[:, 1:]

,value,value2
0,NaN,10.0
1,NaN,40.0
2,30.0,NaN


In [2]:
import pandas as pd 
import numpy as np

true = pd.read_csv('proturn_output_aj_ctrl/hl-data_filtered.csv')
true = np.array(true.pivot(index='ID', columns='t', values='A0').reset_index().drop('ID', axis=1))

mean = pd.read_csv('proturn_output_aj_ctrl/si_mean_4.csv')
mean = np.array(mean.drop('Unnamed: 0', axis=1))

knn = pd.read_csv('proturn_output_aj_ctrl/test_4.csv')

knn = np.array(knn.drop(columns=['Unnamed: 0']))

print(knn.shape, mean.shape, true.shape)

er_mean = np.sum((true - mean)**2)
er_dmi = np.sum((true - knn)**2)
print(er_mean, er_dmi)


(8907, 7) (8907, 7) (8907, 7)
784.3082214125543 1575.1203458071443


In [ ]:
import pandas as pd
import numpy as np
import os

filepath_common = 'proturn_output_'
strains = ['aj', 'balbc', 'c57', 'cej', 'dba', 'fvb']
exp = ['ctrl', 'iso']
true_data_files = 'hl.out'
mask = [1, 2, 3, 4, 5]
output_dir = 'nrmse_k'
folders = [filepath_common + s + '_' + e + '/' for s in strains for e in exp]
os.makedirs(output_dir, exist_ok=True)

def calculate_nrmse(imputed_data, true_data, method):
    # Filter by well-fit peptides
    imputed_data = imputed_data.loc[(imputed_data['R2'] >= 0.8) | (imputed_data['SS'] <= 0.05)]
    imputed_data = imputed_data.loc[(imputed_data['k'] >= 0) & (imputed_data['k'] <= 100)]
    true_data = true_data.loc[(true_data['R2'] >= 0.8)]

    sse = 0
    nf = 0
    for prot in imputed_data['Uniprot'].unique():
        if method == 'dmi':
            # Imputed data for a specific protein
            #imp_prot = imputed_data.loc[imputed_data['Uniprot'] == prot].groupby(['ID', 'Uniprot']).agg({'k': 'mean'}).reset_index()
            # True data for a specific protein
            t = true_data.loc[true_data['Uniprot'] == prot]
            # Filter t so it only contains rows where ID is in imp_prot['ID']
            t = t.loc[t['ID'].isin(imp_prot['ID'])]
            error = np.median(t['k']) - np.median(imp_prot['k'])
            sse += error**2
            nf += np.median(t['k'])**2
        else:
            # Imputed data for a specific protein
            imp_prot = imputed_data.loc[imputed_data['Uniprot'] == prot]
            # True data for a specific protein
            t = true_data.loc[true_data['Uniprot'] == prot]
            # Filter t so it only contains rows where ID is in imp_prot['ID']
            t = t.loc[t['ID'].isin(imp_prot['ID'])]
            error = np.median(t['k']) - np.median(imp_prot['k'])
            sse += error**2
            nf += np.median(t['k'])**2

    return sse, nf

for m in mask:
    sse_mask = 0
    nf_mask = 0
    for f in folders:
        print(f)
        true_path = f"{f}{true_data_files}"
        imputed_path = f"{f}ProTurn_imp_{m}.txt"
        if os.path.exists(true_path) and os.path.exists(imputed_path):
            true = pd.read_csv(true_path, sep='\t', low_memory=False)
            imputed_dmi = pd.read_csv(imputed_path, sep='\t', low_memory=False)
            sse, nf = calculate_nrmse(imputed_dmi, true, 'dmi')
            sse_mask += sse
            nf_mask += nf
        else:
            print(f"Missing file: {true_path} or {imputed_path}")
    if nf_mask > 0:
        nrmse = np.sqrt(sse_mask / nf_mask)
        print(f"NRMSE for mask {m}: {nrmse}")
    else:
        print(f"No valid data for mask {m}")


In [163]:
import pandas as pd
import numpy as np
import os

filepath_common = 'proturn_output_'
strains = ['balbc', 'c57', 'cej', 'dba', 'fvb']
exp = ['ctrl', 'iso']
true_data_files = 'hl.out'
mask = [1, 2, 3, 4, 5]
folders = [filepath_common + s + '_' + e + '/' for s in strains for e in exp]
output_dir = 'nrmse_k'
os.makedirs(output_dir, exist_ok=True)

for m in mask:
    sse_mask = 0
    nf_mask = 0
    for f in folders:
        print(f)
        true_path = f"{f}{true_data_files}"
        imputed_path = f"{f}ProTurn_imp_{m}.txt"
        
        true = pd.read_csv(true_path, sep='\t', low_memory=False)
        true = true.loc[(true['R2'] >= 0.8)]
        imputed_dmi = pd.read_csv(imputed_path, sep='\t', low_memory=False)
        imputed_dmi = imputed_dmi.loc[(imputed_dmi['R2'] >= 0.8)] #| (imputed_dmi['SS'] <= 0.05)]
        imputed_dmi = imputed_dmi.loc[(imputed_dmi['k'] >= 0) & (imputed_dmi['k'] <= 50)]
        
        sse = 0
        nf = 0
        for prot in imputed_dmi.Uniprot.unique():
            if prot not in true.Uniprot.unique():
                pass 
            else: 
                #imputed data for a specific protein
                imp_prot = imputed_dmi.loc[imputed_dmi['Uniprot'] == prot].groupby(['ID', 'Uniprot']).agg({'k': 'mean'}).reset_index()
                #true data for a specific protein
                t = true.loc[true['Uniprot'] == prot][['ID', 'Uniprot','k']]
                #find interesection of IDs between imputed and true data
                imp_prot = imp_prot.loc[imp_prot['ID'].isin(t['ID'])]
                t = t.loc[t['ID'].isin(imp_prot['ID'])]
                #print(imp_prot)
                #print(t)
            
                error = np.median(t['k']) - np.median(imp_prot['k'])
                sse += error**2
                nf += np.median(t['k'])**2
                #print(sse, nf)
        sse_mask += sse
        nf_mask += nf
    nrmse = np.sqrt(sse_mask/nf_mask) 
    print(f"{nrmse} for {m}")
    #with open(f'{output_dir}/nrmse_k.csv', 'a') as f:
    #            f.write(f'{nrmse},{m},dmi\n')
    break

    

proturn_output_balbc_ctrl/


/Users/simhasankar/miniconda3/envs/dmi/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/simhasankar/miniconda3/envs/dmi/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/simhasankar/miniconda3/envs/dmi/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/simhasankar/miniconda3/envs/dmi/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/simhasankar/miniconda3/envs/dmi/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/simhasankar/miniconda3/envs/dmi/lib/python3.1

proturn_output_balbc_iso/


/Users/simhasankar/miniconda3/envs/dmi/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/simhasankar/miniconda3/envs/dmi/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/simhasankar/miniconda3/envs/dmi/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/simhasankar/miniconda3/envs/dmi/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/simhasankar/miniconda3/envs/dmi/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/simhasankar/miniconda3/envs/dmi/lib/python3.1

KeyboardInterrupt: 